In [1]:
# pip install boto3

# Libraries

In [2]:
import pandas as pd
import psycopg2
import json
import boto3

# Configuration

In [3]:
import configparser
config = configparser.ConfigParser()
config.read_file(open(r'C:\Users\hp\Videos\Projects\feranmi\cluster.config'))

In [4]:
KEY = config.get("AWS","KEY")
SECRET=config.get("AWS","SECRET")


DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH", "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH", "DWH_PORT")
DWH_IAM_ROLE_NAME= config.get("DWH","DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

('awsuser', 'Ab!123456', 'myfirstdb')

In [5]:
data = {
    "Param": [
        "DWH_CLUSTER_TYPE", 
        "DWH_NUM_NODES", 
        "DWH_NODE_TYPE", 
        "DWH_CLUSTER_IDENTIFIER", 
        "DWH_DB", 
        "DWH_DB_USER", 
        "DWH_DB_PASSWORD", 
        "DWH_PORT", 
        "DWH_IAM_ROLE_NAME"
    ],
    "Value": [
        DWH_CLUSTER_TYPE, 
        DWH_NUM_NODES, 
        DWH_NODE_TYPE, 
        DWH_CLUSTER_IDENTIFIER, 
        DWH_DB, 
        DWH_DB_USER, 
        DWH_DB_PASSWORD, 
        DWH_PORT, 
        DWH_IAM_ROLE_NAME
    ]
}

df = pd.DataFrame(data)
df


,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Ab!123456
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,s3_fullaccess


In [6]:
ec2 = boto3.resource('ec2',
                    region_name = 'eu-north-1',
                    aws_access_key_id = KEY, 
                    aws_secret_access_key = SECRET)

s3 = boto3.resource('s3',
                    region_name = 'eu-north-1',
                    aws_access_key_id = KEY, 
                    aws_secret_access_key = SECRET)

iam = boto3.client('iam',
                    region_name = 'eu-north-1',
                    aws_access_key_id = KEY, 
                    aws_secret_access_key = SECRET)

redshift = boto3.client('redshift',
                    region_name = 'eu-north-1',
                    aws_access_key_id = KEY, 
                    aws_secret_access_key = SECRET)

In [7]:
# accessing our files inside s3 bucket
bucket = s3.Bucket('grian-enterprise')
log_data_files = [filename for filename in bucket.objects.filter(Prefix='')]
log_data_files

[s3.ObjectSummary(bucket_name='grian-enterprise', key='allevents_pipe.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='allusers_pipe.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='category_pipe.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='date2008_pipe.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='listings_pipe.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='sales_tab.txt'),
 s3.ObjectSummary(bucket_name='grian-enterprise', key='venue_pipe.txt')]

In [8]:
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

In [9]:
roleArn

'arn:aws:iam::590184144492:role/S3_fullaccess'

In [10]:
try:
    response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType=DWH_NODE_TYPE,
    
    DBName = DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword = DWH_DB_PASSWORD,
    
    IamRoles = [roleArn]
        
    )
    
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


In [11]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)

{'Clusters': [{'ClusterIdentifier': 'my-first-redshift',
   'NodeType': 'dc2.large',
   'ClusterStatus': 'available',
   'ClusterAvailabilityStatus': 'Available',
   'MasterUsername': 'awsuser',
   'DBName': 'myfirstdb',
   'Endpoint': {'Address': 'my-first-redshift.cj2bafi24wfg.eu-north-1.redshift.amazonaws.com',
    'Port': 5439},
   'ClusterCreateTime': datetime.datetime(2024, 8, 28, 16, 22, 22, 316000, tzinfo=tzutc()),
   'AutomatedSnapshotRetentionPeriod': 1,
   'ManualSnapshotRetentionPeriod': -1,
   'ClusterSecurityGroups': [],
   'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0b630f23de973212c',
     'Status': 'active'}],
   'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
     'ParameterApplyStatus': 'in-sync'}],
   'ClusterSubnetGroupName': 'default',
   'VpcId': 'vpc-079eef7dddf62bfb5',
   'AvailabilityZone': 'eu-north-1a',
   'PreferredMaintenanceWindow': 'sat:18:30-sat:19:00',
   'PendingModifiedValues': {},
   'ClusterVersion': '1.0',
   'AllowV

In [12]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow =["ClusterIdentifier", "NodeType","ClusterStatus","MasterUsername","DBName","Endpoint","VpcId"]
    x = [(k,v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data = x, columns=["key","value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\hp\AppData\Local\Temp\ipykernel_13216\3032762446.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,key,value
0,ClusterIdentifier,my-first-redshift
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,myfirstdb
5,Endpoint,"{'Address': 'my-first-redshift.cj2bafi24wfg.eu-north-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-079eef7dddf62bfb5


In [13]:
DWH_ENDPOINT =myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

# configuring Amazon VPC

In [14]:
try:
    vpc = ec2.Vpc(id = myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    
    defaultSg.authorize_ingress(
    GroupName=defaultSg.group_name,
    CidrIp ='0.0.0.0/0',
    IpProtocol = 'TCP',
    FromPort = int(DWH_PORT),
    ToPort = int(DWH_PORT)
    )
    
except Exception as e:
    print(e)

An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


# Connecting to Postgres and creating our tables

In [15]:
try:
    conn = psycopg2.connect(host = DWH_ENDPOINT, dbname =DB_NAME,user = DB_USER, password = "Ab!123456",port = 5439)
except psycopg2.Error as e:
    print("Error: could not make connection to the Postgres Database")
    print(e)
    
conn.set_session(autocommit=True)

In [16]:
try:
    cur = conn.cursor()
except Exception as e:
    print('Error: could not get cursor')
    print(e)

In [17]:
try:
    cur.execute("""create table users(
        userid integer not null distkey sortkey,
        username char(8),
        firstname varchar(30),
        lastname varchar(30),
        city varchar(30),
        state char(2),
        email varchar(100),
        phone char(14),
        likesports boolean,
        liketheatre boolean,
        likeconcerts boolean,
        likejazz boolean,
        likeclassical boolean,
        likeopera boolean,
        likerock boolean,
        likevegas boolean,
        likebroadway boolean,
        likemusicals boolean);""")
    
except psycopg2.Error as e:
    print('Error: Issue creating table')
    print(e)


Error: Issue creating table
Relation "users" already exists



In [18]:
try:
    cur.execute("""create table venue(
    venueid smallint not null distkey sortkey,
    venuename varchar(100),
    venuecity varchar(30),
    venuestate char(2),
    venueseats integer);
    """)
    
except psycopg2.Error as e:
    print(e)

Relation "venue" already exists



In [19]:
try:
    cur.execute("""create table category(
        catid smallint not null distkey sortkey,
        catgroup varchar(10),
        catname varchar(10),
        catdesc varchar(50));
        
    create table date(
        dateid smallint not null distkey sortkey,
        caldate date not null,
        day character(3) not null,
        week smallint not null,
        month character(5) not null,
        qtr character(5) not null,
        year smallint not null,
        holiday boolean default('N'));
        
    create table event(
        eventid integer not null distkey,
        venueid smallint not null,
        catid smallint not null,
        dateid smallint not null sortkey,
        eventname varchar(200),
        starttime timestamp);
        
    create table listing(
        listid integer not null distkey,
        sellerid integer not null,
        eventid integer not null,
        dateid smallint not null sortkey,
        numtickets smallint not null,
        priceperticket decimal(8,2),
        totalprice decimal(8,2),
        listtime timestamp);""")
        
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)


# populating our tables with files in s3 bucket

In [21]:
try:
    cur.execute("""
    copy users from 's3://grian-enterprise/allusers_pipe.txt'
    
    credentials 'aws_iam_role=arn:aws:iam::590184144492:role/S3_fullaccess'
    delimiter '|'
    region 'eu-north-1'
    """)
    
    
except psycopg2.Error as e:
    print("error: Issue creating table")
    print(e)

In [25]:
try:
    cur.execute("""
    copy event from 's3://grian-enterprise/allevents_pipe.txt'
    
    credentials 'aws_iam_role=arn:aws:iam::590184144492:role/S3_fullaccess'
    delimiter '|'
    region 'eu-north-1'
    """)
    
    
except psycopg2.Error as e:
    print("error: Issue creating table")
    print(e)

In [26]:
try:
    cur.execute("""
    copy venue from 's3://grian-enterprise/venue_pipe.txt'
    
    credentials 'aws_iam_role=arn:aws:iam::590184144492:role/S3_fullaccess'
    delimiter '|'
    region 'eu-north-1'
    """)
    
    
except psycopg2.Error as e:
    print("error: Issue creating table")
    print(e)

In [27]:
try:
    cur.execute("""
    copy listing from 's3://grian-enterprise/listings_pipe.txt'
    
    credentials 'aws_iam_role=arn:aws:iam::590184144492:role/S3_fullaccess'
    delimiter '|'
    region 'eu-north-1'
    """)
    
    
except psycopg2.Error as e:
    print("error: Issue creating table")
    print(e)

In [22]:
try:
    cur.execute("""
    select * from users;
    """)
    
except psycopg2.Error as e:
    print("error: Issue creating table")
    print(e)

In [23]:
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    break

(1, 'JSG99FHE', 'Rafael', 'Taylor', 'Kent', 'WA', 'Etiam.laoreet.libero@sodalesMaurisblandit.edu', '(664) 602-4412', True, True, None, False, True, None, None, True, False, True)


# Close connection

In [28]:
try:
    conn.close()
    
except psycopg2.Error as e:
    print(e)

In [30]:
redshift.delete_cluster(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot= True)

{'Cluster': {'ClusterIdentifier': 'my-first-redshift',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsuser',
  'DBName': 'myfirstdb',
  'Endpoint': {'Address': 'my-first-redshift.cj2bafi24wfg.eu-north-1.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2024, 8, 28, 16, 22, 22, 316000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0b630f23de973212c',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-079eef7dddf62bfb5',
  'AvailabilityZone': 'eu-north-1a',
  'PreferredMaintenanceWindow': 'sat:18:30-sat:19:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  '